In [ ]:
import numpy as np
import csv
import torch
import torch.nn as nn
from torch.nn import DataParallel
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetTemperature, nvmlShutdown, NVML_TEMPERATURE_GPU
import matplotlib.pyplot as plt
import os
import time
import datetime
import itertools
from sklearn.model_selection import train_test_split
import gc
import pynvml
import optuna
import random
optuna.logging.disable_default_handler()

rng = np.random.RandomState(1234)
random_state = 42
start_time=time.time()



def timecount():
    end_time=time.time()
    elapsed_time=end_time-start_time
    hours, rem = divmod(elapsed_time, 3600)  # 3600秒 = 1時間
    minutes, seconds = divmod(rem, 60)  # 60秒 = 1分
    # 経過時間の表示
    print(f"処理にかかった時間: {int(hours)}時間 {int(minutes)}分 {seconds:.2f}秒")
def set_all_seeds(seed):
    """
    Set seeds for all the major random number generators to ensure reproducibility.
    """
    random.seed(seed)  # Python標準の乱数
    np.random.seed(seed)  # NumPyの乱数
    torch.manual_seed(seed)  # PyTorchのCPU乱数
    torch.cuda.manual_seed_all(seed)  # PyTorchのGPU乱数

    # 再現性のための設定
    torch.backends.cudnn.deterministic = True  # 決定的な演算を有効化
    torch.backends.cudnn.benchmark = False  # ベンチマークを無効化



In [14]:
class BatchNorm(nn.Module):
    def __init__(self, shape, epsilon=np.float32(1e-5)):
        super().__init__()
        self.gamma = nn.Parameter(torch.tensor(np.ones(shape, dtype='float32')))
        self.beta = nn.Parameter(torch.tensor(np.zeros(shape, dtype='float32')))
        self.epsilon = epsilon

    def forward(self, x):
        mean = torch.mean(x, (0, 2, 3), keepdim=True)  # WRITE ME
        std = torch.std(x, (0, 2, 3), keepdim=True)  # WRITE ME
        x_normalized = (x - mean) / (std**2 + self.epsilon)**0.5  # WRITE ME
        return self.gamma * x_normalized + self.beta  # WRITE ME
    
class Dropout(nn.Module):
    """
    http://arxiv.org/abs/1207.0580
    """
    def __init__(self, dropout_ratio=0.5):
        super().__init__()
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x):
        # 学習時はdropout_ratio分だけ出力をシャットアウト
        if self.training:
            self.mask = torch.rand(*x.size()) > self.dropout_ratio
            return x * self.mask.to(x.device)
        # 推論時は出力に`1.0 - self.dropout_ratio`を乗算することで学習時の出力の大きさに合わせる
        else:
            return x * (1.0 - self.dropout_ratio)
        
class Conv(nn.Module):
    def __init__(self, filter_shape, function=lambda x: x, stride=(1, 1), padding=0):
        super().__init__()
        # Heの初期化
        # filter_shape: (出力チャンネル数)x(入力チャンネル数)x(縦の次元数)x(横の次元数)
        fan_in = filter_shape[1] * filter_shape[2] * filter_shape[3]
        fan_out = filter_shape[0] * filter_shape[2] * filter_shape[3]

        self.W = nn.Parameter(torch.tensor(rng.normal(
                        0,
                        np.sqrt(2/fan_in),
                        size=filter_shape
                    ).astype('float32')))

        # バイアスはフィルタごとなので, 出力フィルタ数と同じ次元数
        self.b = nn.Parameter(torch.tensor(np.zeros((filter_shape[0]), dtype='float32')))

        self.function = function  # 活性化関数
        self.stride = stride  # ストライド幅
        self.padding = padding  # パディング

    def forward(self, x):
        u = F.conv2d(x, self.W, bias=self.b, stride=self.stride, padding=self.padding)
        return self.function(u)
    
class Pooling(nn.Module):
    def __init__(self, ksize=(1, 1), stride=(1, 1), padding=0):
        super().__init__()
        self.ksize = ksize  # カーネルサイズ
        self.stride = stride  # ストライド幅
        self.padding = padding  # パディング

    def forward(self, x):
        return F.max_pool2d(x, kernel_size=self.ksize, stride=self.stride, padding=self.padding)

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.size()[0], -1)
    
class Dense(nn.Module):
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        # Heの初期化
        # in_dim: 入力の次元数，out_dim: 出力の次元数       
        self.W = nn.Parameter(torch.tensor(rng.normal(
                        0,
                        np.sqrt(2/in_dim),
                        size=(in_dim, out_dim)
                    ).astype('float32')))
        
        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]).astype('float32')))
        self.function = function
        
    def forward(self, x):
        return self.function(torch.matmul(x, self.W) + self.b)
    
class Activation(nn.Module):

    def __init__(self, function=lambda x: x):
        super().__init__()
        self.function = function

    def __call__(self, x):
        return self.function(x)

def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))


In [15]:
#prese_1pulse_dataloader_save.pyにて作成したデータを読み込む。
# 保存したテンソルを読み込み
X_train, y_train = torch.load('/mnt/sdb/ywatanabe/CNN_dataset/transwave/1pulse/dataset_train_200260.pt')
X_valid, y_valid = torch.load('/mnt/sdb/ywatanabe/CNN_dataset/transwave/1pulse/dataset_valid_200260.pt')


# PyTorchのテンソルをNumPy配列に変換
X_train_np, y_train_np = X_train.numpy(), y_train.numpy()
X_valid_np, y_valid_np = X_valid.numpy(), y_valid.numpy()

# train_test_splitでデータを分割
X_train_np, X_no_use_np, y_train_np, y_no_use_np = train_test_split(X_train_np, y_train_np, test_size=0.8, random_state=42)
X_valid_np, X_no_use_np, y_valid_np, y_no_use_np = train_test_split(X_valid_np, y_valid_np, test_size=0.8, random_state=42)

# NumPy配列を再びPyTorchテンソルに戻す
X_train, y_train = torch.tensor(X_train_np), torch.tensor(y_train_np)
X_valid, y_valid = torch.tensor(X_valid_np), torch.tensor(y_valid_np)


# TensorDataset と DataLoader を再作成
dataset_train = TensorDataset(X_train, y_train)
dataset_valid = TensorDataset(X_valid, y_valid)





In [ ]:

def objective(trial):
        print(trial.number)
        set_all_seeds(trial.number)
                
        device = torch.device("cuda:1")  # メインのGPUを指定    optimizer = optim.SGD(conv_net.parameters(), lr=lr)
        avg_train_mse=0.0
        avg_valid_mse=0.0              
        avg_train_mse_list=[]
        avg_valid_mse_list=[]

        # 相関係数の計算
        correlation_coefficient_list=[]
        mae_list = []
        relative_error_list = []    
        try:

                #畳み込み層の数
                filter_number = trial.suggest_int("filter_number", 64, 128, step=32)
                y_length = 15
                x_length = 3112
                x_conv = int(trial.suggest_int("x_conv", 3, 5))
                x_stride = int(trial.suggest_int("x_stride", 1, 2))
                conv_padding = 0
                y_conv=1
                y_stride=1
                y_pool=1
                x_pool = int(trial.suggest_int("x_pool", 4, 8))
                lr = trial.suggest_float('learning rate', 0.00001, 0.0005, log=True)
                n_epochs = 20
                Dense2_length = int(trial.suggest_int("mid_units", 64, 128, step=32))
                batch_size = int(trial.suggest_int("batch_size", 32, 128, step=32))

                loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
                loader_valid = DataLoader(dataset=dataset_valid, batch_size=batch_size, shuffle=False)
                # DataLoaderのサイズ（バッチ数）
                num_batches = len(loader_train)
                print(f'Number of batches in dataloader_train: {num_batches}')

                # データセット全体のサイズ（サンプル数）
                dataset_size = len(dataset_train)
                print(f'Number of samples in dataset_train: {dataset_size}')
                #del X_train,X_valid,y_train,y_valid,dataset_train,dataset_valid,X_train_np, X_no_use_np, y_train_np, y_no_use_np,X_valid_np, y_valid_np,
            
                
                
                path_in='/mnt/sdb/ywatanabe/Saved_png_after1130/transwave/20percent_input_200260/1pulse'
                
                path_in=os.path.join(path_in)
                print(f"Training with parameters: x_length={x_length}, x_conv={x_conv}, x_stride={x_stride}, x_pool={x_pool}, "
                        f"filter_number={filter_number}, lr={lr}, n_epochs={n_epochs}, batch_size={batch_size}")
                Title="x_conv"+str(x_conv)+"x_stride"+str(x_stride)+"x_pool"+str(x_pool)+"Filter_number"+str(filter_number)+"lr"+str(lr)+"batchsize"+str(batch_size)
                path_image=os.path.join("/mnt/sdb/ywatanabe/Saved_png_after1130/transwave/20percent_input/1pulse/mse",Title+".png")
                
                B1_x_length=int((x_length+2*conv_padding-x_conv)//x_stride+1)
                B2_x_length=int((B1_x_length//x_pool+2*conv_padding-x_conv)//x_stride+1)
                B1_y_length=int((y_length+2*conv_padding-y_conv)//y_stride+1)
                B2_y_length=int((B1_y_length//y_pool+2*conv_padding-y_conv)//y_stride+1)   
                Dense_length=int((B2_x_length//x_pool)*(B2_y_length//y_pool)*filter_number)
                conv_net = nn.Sequential(
                Conv(filter_shape=(filter_number, 1, y_conv, x_conv), stride=(1, x_stride), padding=conv_padding),        # 画像の大きさ：1x4012x1 -> 1x3996x64  # WRITE ME(入出力の画像サイズ）
                BatchNorm((filter_number, B1_y_length, B1_x_length)),    #15pulse分あるため
                Activation(F.relu),
                Pooling(ksize=(1, x_pool), stride=(1, x_pool), padding=conv_padding),            # 1x3996x64 -> 1x999x64  # WRITE ME(入出力の画像サイズ）
                Conv(filter_shape=(filter_number, filter_number, y_conv, x_conv),stride=(1, x_stride)),       # 1x999x64 -> 1x983x64  # WRITE ME(入出力の画像サイズ）
                BatchNorm((filter_number, B2_y_length, B2_x_length)),
                Activation(F.relu),
                Pooling(ksize=(1, x_pool), stride=(1, x_pool)),            # 1x983x64 -> 1x250x64????なぜかこうなってる  # WRITE ME(入出力の画像サイズ）
                Flatten(),
                Dense(Dense_length, Dense2_length, F.relu),  # 1
                Dense(Dense2_length, 1)
                )
                
                conv_net = conv_net.to(device)  # モデルを指定したGPUに移動
                #conv_net = DataParallel(conv_net, device_ids=[ 1, 0, 2], output_device=1)  # GPU1を主デバイスに設定
                criterion = nn.MSELoss()
                #optimizer = get_optimizer(trial, conv_net)
                optimizer = optim.SGD(conv_net.parameters(), lr=lr)

                
                for epoch in range(n_epochs):
                        #train
                        #memoricount()    
                        conv_net.train()  # 訓練モードにする]
                        total_trainloss = 0.0
                        for x, t in loader_train:
                                conv_net.zero_grad()  # 勾配の初期化
                                x = x.to(device).float()  # テンソルをGPUに移動し、データ型をfloat32に変換
                                t = t.to(device).float()
                                y = conv_net.forward(x)  # 順伝播
                                y = y.squeeze(dim=1)
                                loss = criterion(y, t)
                                loss.backward()  # 誤差の逆伝播
                                optimizer.step()  # パラメータの更新
                                total_trainloss += loss.item()
                        avg_train_mse = total_trainloss / len(loader_train)
                        avg_train_mse_list.append(avg_train_mse)

                        #test
                        y_list = []
                        t_list = []
                        total_validloss = 0.0

                        conv_net.eval()
                        with torch.no_grad():
                                for x, t in loader_valid:
                                        x = x.to(device)  # テンソルをGPUに移動

                                        t = t.to(device)
                                        y = conv_net.forward(x)  # 順伝播
                                        y = y.squeeze(dim=1)

                                        loss = criterion(y, t)
                                        y_list.extend(y)
                                        t_list.extend(t)
                                                
                                        # バッチごとの損失を蓄積
                                        total_validloss += loss.item()
                        avg_valid_mse = total_validloss / len(loader_valid)
                        avg_valid_mse_list.append(avg_valid_mse)
                        if np.isnan(avg_train_mse):
                                print("Loss became NaN. Stopping training.")
                                
                                break
                        if np.isnan(avg_valid_mse):
                                print("Loss became NaN. Stopping training.")
                                break
                        if epoch%5==4:
                                timecount()
                                print('EPOCH: {}, Train [Loss: {:.6f} ], Valid [Loss: {:.6f} ]'.format(
                                        epoch,
                                        avg_train_mse,
                                        avg_valid_mse
                                        ))

                        # y_list と t_list をそれぞれ numpy 配列に変換
                        y_array = np.array([y.cpu().numpy() for y in y_list])
                        t_array = np.array([t.cpu().numpy() for t in t_list])

                        # グラフを描画
                        plt.figure(figsize=(10, 10))
                        plt.scatter(t_array, y_array, alpha=0.08)  # 散布図として描画
                        #plt.plot(t_array, y_array, linestyle='-', alpha=0.2)  # 折れ線としても表示することで変化を見やすく
                        plt.plot([0, 0.2], [0, 0.2], color='red', linestyle='--', label='y = x')  # 基準線
                        plt.xlim(0,0.2)
                        plt.ylim(0,0.2)
                        plt.xlabel("t_list (Ground Truth)")
                        plt.ylabel("y_list (Predicted)")
                        Title="x_conv"+str(x_conv)+"x_stride"+str(x_stride)+"x_pool"+str(x_pool)+"Filter_number"+str(filter_number)+"epoch"+str(epoch)+"lr"+str(lr)
                        plt.title(Title)
                        plt.grid(True)
                        path_image=os.path.join(path_in,"plot",Title+".png")
                        plt.savefig(path_image, format="png", dpi=300)
                        #plt.show()
                        plt.close()
                        # グラフを描画

                        plt.figure(figsize=(12, 10))
                                # ヒストグラムを2Dカラーマップで描画
                        plt.hist2d(t_array.flatten(), y_array.flatten(), bins=40, range=[[0, 0.2], [0, 0.2]], cmap='viridis', cmin=0, density=True)
                        plt.colorbar(label="Frequency")  # カラーバーを追加    
                        plt.plot([0, 0.2], [0, 0.2], color='red', linestyle='--', label='y = x')  # 基準線
                        plt.xlim(0,0.2)
                        plt.ylim(0,0.2)
                        plt.xlabel("t_list (Ground Truth)")
                        plt.ylabel("y_list (Predicted)")
                        Title="x_conv"+str(x_conv)+"x_stride"+str(x_stride)+"x_pool"+str(x_pool)+"Filter_number"+str(filter_number)+"epoch"+str(epoch)+"lr"+str(lr)+"midlength"+str(Dense2_length)
                        plt.title(Title)
                        plt.grid(True)
                        path_image=os.path.join(path_in,"colormap",Title+".png")
                        plt.savefig(path_image, format="png", dpi=300)
                        #plt.show()
                        plt.close()

                        correlation_coefficient = np.corrcoef(y_array.flatten(), t_array.flatten())[0, 1]# 相関係数の計算
                        mae = np.mean(np.abs(y_array - t_array))# MAE (Mean Absolute Error) の計算
                        relative_error = np.mean(np.abs(y_array - t_array) / (np.abs(t_array) + 1e-8))  # 相対誤差 (Relative Error) の計算# 真の値が0の場合はゼロ除算を防ぐために条件を追加
                        correlation_coefficient_list.append(correlation_coefficient)
                        mae_list.append(mae)
                        relative_error_list.append(relative_error)
                if not np.isnan(avg_valid_mse):

                        plt.figure(figsize=(10, 10))
                        plt.plot(range(1,n_epochs), avg_train_mse_list[1:])
                        plt.plot(range(1,n_epochs), avg_valid_mse_list[1:], c='#00ff00')
                        #print(avg_valid_mse_list)
                        #plt.xlim(0, n_epochs)
                        #plt.ylim(0, 2.5)
                        plt.xticks(np.arange(0, 20, 5))

                        plt.xlabel('EPOCH')
                        plt.ylabel('LOSS')
                        plt.legend(['train loss', 'test loss'])
                        Title="x_conv"+str(x_conv)+"x_stride"+str(x_stride)+"x_pool"+str(x_pool)+"Filter_number"+str(filter_number)+"lr"+str(lr)+"batchsize"+str(batch_size)+"mid_length"+str(Dense2_length)
                        plt.title(Title)
                        path_image=os.path.join(path_in,"mse",Title+".png")
                        plt.savefig(path_image, format="png", dpi=300)
                        plt.close()

                        # 保存するデータ
                        data = [
                                {"epoch": epoch + 1,
                                "correlation_coefficient": correlation_coefficient_list[epoch],
                                "mae": mae_list[epoch],
                                "train_mse": avg_train_mse_list[epoch],
                                "valid_mse": avg_valid_mse_list[epoch],
                                "relative_error": relative_error_list[epoch]}
                                for epoch in range(n_epochs)
                        ]
                        # 保存するCSVファイル名
                        path_csv=os.path.join(path_in,"csv",Title+".csv")
                        # CSVファイルに書き込み
                        with open(path_csv, mode="w", newline="", encoding="utf-8") as file:
                                # ヘッダーを作成
                                fieldnames = ["epoch", "correlation_coefficient", "mae", "train_mse","valid_mse","relative_error"]
                                writer = csv.DictWriter(file, fieldnames=fieldnames)
                                # ヘッダーを書き込み
                                writer.writeheader()
                                # データを書き込み
                                writer.writerows(data)

                        #wait_for_gpu_temperature(threshold=65,  check_interval=3*60)
                        print(datetime.datetime.now())
                        time.sleep(1*60)
        except RuntimeWarning as e:
                print(f"RuntimeWarning: {e}")
                avg_valid_mse = np.nan
        except Exception as e:
                print(f"Unexpected error: {e}")
                avg_valid_mse = np.nan


        return avg_valid_mse


In [ ]:

db_path = "/mnt/sdb/ywatanabe/CNN_dataset/transwave/optuna_data/1pulse_20percent_input_200260.db"
storage_name = f"sqlite:///{db_path}"


TRIAL_SIZE = 100
study = optuna.create_study(storage=storage_name, study_name="1pulse_20percent_input", load_if_exists=True)
study.optimize(objective, n_trials=TRIAL_SIZE)
# optuna-dashboard sqlite:////mnt/sdb/ywatanabe/CNN_dataset/transwave/optuna_data/1pulse_20percent_input.db --port 6002
# 上記分をターミナルで実行すればよい

Number of batches in dataloader_train: 195
Number of samples in dataset_train: 12456
  Used Memory: 5.46 GB
Training with parameters: x_length=3112, x_conv=3, x_stride=1, x_pool=4, filter_number=96, lr=0.00025872714757899004, n_epochs=20, batch_size=64
処理にかかった時間: 0時間 3分 5.33秒
EPOCH: 4, Train [Loss: 0.001045 ], Valid [Loss: 0.001046 ]
処理にかかった時間: 0時間 6分 8.51秒
EPOCH: 9, Train [Loss: 0.000974 ], Valid [Loss: 0.000975 ]
処理にかかった時間: 0時間 9分 11.07秒
EPOCH: 14, Train [Loss: 0.000939 ], Valid [Loss: 0.000947 ]
処理にかかった時間: 0時間 12分 13.77秒
EPOCH: 19, Train [Loss: 0.000917 ], Valid [Loss: 0.000936 ]
2024-12-03 14:28:41.961144
Number of batches in dataloader_train: 195
Number of samples in dataset_train: 12456
  Used Memory: 5.46 GB
Training with parameters: x_length=3112, x_conv=3, x_stride=1, x_pool=7, filter_number=128, lr=0.00025219833211220875, n_epochs=20, batch_size=64
処理にかかった時間: 0時間 16分 31.23秒
EPOCH: 4, Train [Loss: 0.001287 ], Valid [Loss: 0.001290 ]
処理にかかった時間: 0時間 19分 47.16秒
EPOCH: 9, Train [L

Trial 327 failed with parameters: {'filter_number': 128, 'x_conv': 4, 'x_stride': 1, 'x_pool': 4, 'learning rate': 0.00040510546706859344, 'mid_units': 96, 'batch_size': 64} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/mnt/sdb/ywatanabe/yuwatanabe/yuvenv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_753008/826425256.py", line 223, in objective
    time.sleep(1*60)
KeyboardInterrupt
Trial 327 failed with value None.


KeyboardInterrupt: 

: 

In [ ]:
print(f'Best value: {study.best_value}')
print(f'Best param: {study.best_params}')

In [ ]:
import sqlite3
#db_path = "/mnt/sdb/ywatanabe/CNN_dataset/optuna_data/example.db"
if False:

    # データベースに接続
    conn = sqlite3.connect(db_path)

    # データベース内のテーブルを確認
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(cursor.fetchall())

    # 特定のテーブル内容を確認（例: `trials` テーブル）
    cursor.execute("SELECT * FROM trials;")
    print(cursor.fetchall())

    # 接続を閉じる
    conn.close()
# optuna-dashboard sqlite:////mnt/sdb/ywatanabe/CNN_dataset/reflectedwave/optuna_data/20percent_input.db --port 8000
# 上記分をターミナルで実行すればよい